In [1]:
import pandas as pd
import functools as ft
import numpy as np
from os import listdir
from os.path import isfile, join
import utm

import sys

sys.path.append("..")
from tools import data_path, output_path, cache_path

In [2]:
# Data was crawled with a custom crawler that uses the API provided by the saxonian authorities

In [3]:
mypath = data_path + "saxony/"
onlyfiles = [mypath + f for f in listdir(mypath) if isfile(join(mypath, f))]
meta = pd.read_csv(data_path + "saxony_crawler/Stammdaten.csv")

In [4]:
holder = []
for x in onlyfiles:
    d = pd.read_csv(x)
    if "Q" not in d.columns: 
        print(x)
    else:
        stack = d[["Q_timestamp", "Q"]].dropna()
        stack["Q_timestamp"] = pd.to_datetime(stack["Q_timestamp"].str[:-9])
        stack.drop_duplicates("Q_timestamp", inplace=True)
        stack.rename(columns={"Q":x.split("/")[-1].split("_")[-2]}, inplace=True)
        stack.index = stack["Q_timestamp"]
        stack.drop(columns="Q_timestamp", inplace=True)
        holder.append(stack)

/home/datasets4/stein/rivers/raw_data/saxony/Hainsberg6_VereinigteWeißeritz_551190_Ziel.csv
/home/datasets4/stein/rivers/raw_data/saxony/Ziegenhain_Ketzerbach_552000_Ziel.csv


/tmp/ipykernel_92213/316021609.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(x)


/home/datasets4/stein/rivers/raw_data/saxony/Hagenwerder3_LausitzerNeiße_660130_Ziel.csv
/home/datasets4/stein/rivers/raw_data/saxony/Schwarzenberg_Schwarzwasser_563780_Ziel.csv


In [5]:
df_final = ft.reduce(lambda left, right: pd.merge(left, right,how="outer", left_index=True, right_index=True), holder)
df_final.index.name = "datetime"
df_final.columns = [x + "_s" for x in df_final.columns]

In [6]:
[x for x in df_final.columns if "501" in x]

['577501_s', '550110_s', '550190_s']

In [7]:
# We got all stations. 
[x for x in df_final.columns if int(x[:-3]) not in meta["Pegelkennziffer"].values]

['554210_s',
 '583235_s',
 '568133_s',
 '550290_s',
 '554570_s',
 '554120_s',
 '551890_s',
 '562070_s',
 '662011_s',
 '564531_s',
 '552401_s',
 '551208_s',
 '662001_s',
 '530020_s',
 '552110_s',
 '583271_s',
 '564611_s',
 '551302_s',
 '577501_s',
 '565980_s',
 '560021_s',
 '551601_s',
 '566085_s',
 '554665_s',
 '660001_s',
 '583302_s',
 '560301_s',
 '550110_s',
 '562014_s',
 '563290_s',
 '576420_s',
 '563890_s',
 '562115_s',
 '550913_s',
 '576440_s',
 '551351_s',
 '560000_s',
 '551000_s',
 '550800_s',
 '583223_s',
 '564201_s',
 '582080_s',
 '576635_s',
 '568160_s',
 '578091_s',
 '552210_s',
 '582032_s',
 '576401_s',
 '567435_s',
 '564290_s',
 '583251_s',
 '583280_s',
 '582005_s',
 '567590_s',
 '551820_s',
 '552305_s',
 '554520_s',
 '551420_s',
 '564120_s',
 '567890_s',
 '583350_s',
 '550390_s',
 '563790_s',
 '577068_s',
 '567390_s',
 '550710_s',
 '567000_s',
 '582820_s',
 '566010_s',
 '562090_s',
 '550190_s',
 '662022_s',
 '576421_s',
 '554550_s',
 '564620_s',
 '567310_s',
 '554510_s',

In [8]:
meta_data_saxony = meta.loc[
    meta["Pegelkennziffer"].isin(np.array([x[:-2] for x in df_final.columns]).astype(int)),
    [
        "Pegelkennziffer",
        "Pegelname",
        "Gewaesser",
        "Ostwert",
        "Nordwert",
        "Lage am Wasserlauf (km)",
        "Pegelnullpunkt (m)",
        "Gebietskennzahl"
    ],
]

In [9]:
# No issues here.
meta_data_saxony.isnull().sum()

Pegelkennziffer            0
Pegelname                  0
Gewaesser                  0
Ostwert                    0
Nordwert                   0
Lage am Wasserlauf (km)    0
Pegelnullpunkt (m)         0
Gebietskennzahl            0
dtype: int64

### Transform to general format

In [10]:
# saxony parsing
for ind, line in meta_data_saxony.iterrows():
    try:
        meta_data_saxony.loc[ind, ["Ostwert", "Nordwert"]] = utm.to_latlon(
            line["Ostwert"], line["Nordwert"], 33, "U"
        )
    except:
        print(ind)

In [11]:
meta_data_saxony.index = meta_data_saxony["Pegelkennziffer"].values
meta_data_saxony = meta_data_saxony[["Gewaesser",	"Ostwert",	"Nordwert",	"Lage am Wasserlauf (km)",	"Pegelnullpunkt (m)"]]
meta_data_saxony.columns = ["R", "X", "Y", "D", "H"]
meta_data_saxony.index.name = "ID"

In [12]:
meta_data_saxony[meta_data_saxony["R"] == "Rosenbach"]

,R,X,Y,D,H
ID,,,,,
583020,Rosenbach,51.037761,14.451852,1.0,294.033
577068,Rosenbach,50.476818,12.070145,1.2,351.453


In [13]:
meta_data_saxony

,R,X,Y,D,H
ID,,,,,
576401,Weiße Elster,50.329416,12.258146,225.4,436.104
578091,Parthe,51.305912,12.565887,30.9,125.045
564531,Zwönitz,50.791358,12.927491,0.3,315.592
551310,Wilde Weißeritz,50.805328,13.606826,33.3,527.367
567590,Sehma,50.558317,12.998784,6.5,560.914
...,...,...,...,...,...
660100,Lausitzer Neiße,50.896553,14.836702,194.2,225.150
662022,Mandau,50.894758,14.793497,2.5,232.361
576635,Weiße Elster,51.221760,12.273708,57.2,118.654


In [14]:
# 2 rivers with the same name. 
meta_data_saxony.loc[583020, "R"] = "Rosenbach (Spree)" #("One is a different one")

In [15]:
meta_data_saxony["QD"] = -1
meta_data_saxony["QH"] = -1
meta_data_saxony["QX"] = -1
meta_data_saxony["QY"] = -1
meta_data_saxony["QR"] = -1

In [16]:
meta_data_saxony.loc[~(meta_data_saxony["X"].isnull()), "QX"] = 0
meta_data_saxony.loc[~(meta_data_saxony["Y"].isnull()), "QY"] = 0
meta_data_saxony.loc[~(meta_data_saxony["D"].isnull()), "QD"] = 0
meta_data_saxony.loc[~(meta_data_saxony["R"].isnull()), "QR"] = 0
meta_data_saxony.loc[~(meta_data_saxony["H"].isnull()), "QH"] = 0

In [17]:
meta_data_saxony.loc[583020, "QR"]  =  1

In [18]:
meta_data_saxony

,R,X,Y,D,H,QD,QH,QX,QY,QR
ID,,,,,,,,,,
576401,Weiße Elster,50.329416,12.258146,225.4,436.104,0,0,0,0,0
578091,Parthe,51.305912,12.565887,30.9,125.045,0,0,0,0,0
564531,Zwönitz,50.791358,12.927491,0.3,315.592,0,0,0,0,0
551310,Wilde Weißeritz,50.805328,13.606826,33.3,527.367,0,0,0,0,0
567590,Sehma,50.558317,12.998784,6.5,560.914,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
660100,Lausitzer Neiße,50.896553,14.836702,194.2,225.150,0,0,0,0,0
662022,Mandau,50.894758,14.793497,2.5,232.361,0,0,0,0,0
576635,Weiße Elster,51.221760,12.273708,57.2,118.654,0,0,0,0,0


In [19]:
meta_data_saxony.to_csv(output_path + "saxony_meta_data.csv")
df_final.to_csv(output_path + "saxony_processed.csv")